In [1]:
from urllib.request import urlopen
from lxml import etree
import re
import xlwt
import xlrd
from xlutils.copy import copy
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests
import json
import time
import sys
from multiprocessing import Pool

In [8]:
# 使用了各种方法，去请求网页数据，都会被网站屏蔽。
# 所以这里采用自动化测试点击下一页来获取每页数据
browser = webdriver.Chrome()

In [9]:
browser.get("http://data.10jqka.com.cn/financial/yjgg/###")

In [10]:
# wait = WebDriverWait(browser, 10)

In [11]:
# workbook = copy(xlrd.open_workbook("data/上市公司.xls"))
workbook = xlwt.Workbook()

In [12]:
heads = ['StockCode','StockName',\
        'Trade','PB','PE_S','PE_D','NetProfitDes','ROE','GrossProfitRate','CapitalStock','ScaleShareType']
# 获取股票数据
def getStockDataByType(stock_type):
    # StockCode：股票代码
    # StockName：股票简称
    # Trade：行业
    # PB：市净率
    # PE_S：静态市盈率
    # PE_D：动态市盈率
    # NetProfitDes：净利润描述（包括净利润和增长比率）
    # ROE：净资产收益率
    # GrossProfitRate：毛利率
    # CapitalStock：股本
    # ScaleShareType：类型（大盘股、小盘股）
    
    # Proceeds：营业收入
    # ProceedsYOY：营业收入-同比增长
    # ProceedsQOQ：营业收入-环比增长
    # NetProfit：净利润
    # NetProfitYOY：净利润-同比增长
    # NetProfitQOQ：净利润-环比增长
    # EarningsPerShare：每股收益
    # NetAssetValuePerShare：每股净资产
    # MoneyFlowPerShare：每股现金流量
    # GrossProfitRate：销售毛利率
    boards = browser.find_elements_by_class_name("J-board-item")
    
    for i,board in enumerate(boards):
        board_name =board.text
        if board_name == stock_type:
            sheet = workbook.add_sheet(board_name) 
            for h in range(len(heads)):
                sheet.write(0, h, heads[h]) 
            board.click()
            time.sleep(3)
            print("正在获取板块："+board_name+"...")
            total_page = int(browser.find_element_by_class_name("page_info").text.split("/")[1])
#             total_page = 2
            for page in range(1,total_page+1):
                comapnies = browser.find_elements_by_class_name("J_showCanvas")
                page_size = len(comapnies)
#                 print("第"+str(page)+"页...")
                for j,company in enumerate(comapnies):
                    printProgress("第"+str(page)+"页-"+"第"+str(j)+"条...")
                    code = browser.find_element_by_xpath('//*[@id="J-ajax-main"]/table/tbody/tr['+str(j+1)+']/td[2]/a')
                    row = (page-1)*50+1+j
                    sheet.write(row,0,code.text)
                    sheet.write(row,1,company.text)
                    getBaseData(code.text,row,2,sheet)
                
                page_btns = browser.find_elements_by_class_name("changePage")
                subscript = 0
                if page != total_page+1:
                    for page_btn in page_btns:
                        if page_btn.text == "下一页":
                            subscript = page_btns.index(page_btn)
                    page_btns[subscript].click()
                    time.sleep(3)
#                         wait.until(EC.element_to_be_clickable(By.))
    browser.close()
    browser.quit()

def getBaseData(code,row,index,sheet):
    try:
        #设置用户代理，不然会被网站屏蔽
        headers = {"user-agent":"PostmanRuntime/7.13.0"}
        response_1 = requests.get("http://basic.10jqka.com.cn/"+code+"/company.html",headers = headers)
        response_1.encoding = 'gbk'
        e_1 = etree.HTML(response_1.text)
        # 获取行业
        try:
            trade = e_1.xpath('//*[@id="detail"]/div[2]/table/tbody/tr[2]/td[2]/span')[0]
            sheet.write(row,index,trade.text)
        except IndexError:
            trade = e_1.xpath('//*[@id="detail"]/div[3]/table/tbody/tr[2]/td[2]/span')[0]
            sheet.write(row,index,trade.text)
            
#         #上市时间
#         ttm = e_1.xpath('//*[@id="publish"]/div[2]/table/tbody/tr[2]/td[1]/span')[0]
#         sheet.write(row,index+1,ttm.text)

        # 获取市净率
        response_2 = requests.get("http://basic.10jqka.com.cn/"+code+"/",headers = headers)
        response_2.encoding = 'gbk'
        e_2 = etree.HTML(response_2.text)
        pb = e_2.xpath('//*[@id="sjl"]')[0]
        sheet.write(row,index+1,pb.text)
        # 获取市盈率
        # 静态市盈率
        pe_s = e_2.xpath('//*[@id="jtsyl"]')[0]
        sheet.write(row,index+2,pe_s.text)
        # 动态市盈率
        pe_d = e_2.xpath('//*[@id="dtsyl"]')[0]
        sheet.write(row,index+3,pe_d.text)
        # 净利润
        np_des = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[3]/td[2]/span[2]')[0]
        sheet.write(row,index+4,np_des.text)

        # 净资产收益率
        roe = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[4]/td[3]/span[2]')[0]
        sheet.write(row,index+5,roe.text)
        
        #毛利率
        gpr = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[4]/td[2]/span[2]')[0]
        sheet.write(row,index+6,gpr.text)
        #股本
        cs = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[2]/td[4]/span[2]/text()')[0]
        sheet.write(row,index+7,str(cs))
        #类型
        sst = e_2.xpath('//*[@id="profile"]/div[2]/table[2]/tbody/tr[1]/td[4]/span[2]')[0]
        sheet.write(row,index+8,sst.text)

    except ConnectionResetError:
        print(code)
def printProgress(content):
    sys.stdout.write("\r" +content)
    sys.stdout.flush()

In [13]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
dir_path = 'D:/wt/PythonWorkSpace/DataAnalysis/Stock_Companies/data/'

# getStockDataByType("沪市A股")
# workbook.save(dir_path+'上市公司沪市.xls')

getStockDataByType("深市A股")
workbook.save(dir_path+'上市公司深市.xls')

# getStockDataByType("中小板")
# workbook.save(dir_path+'上市公司中小板.xls')

# getStockDataByType("创业板")
# workbook.save(dir_path+'上市公司创业板.xls')

正在获取板块：深市A股...
第10页-第15条...

In [29]:
headers = {"user-agent":"PostmanRuntime/7.13.0"}
response = requests.get("http://basic.10jqka.com.cn/000687/company.html",headers = headers)
# response.encoding = response.apparent_encoding
response.encoding = 'gbk'

# response.apparent_encoding

In [31]:
e = etree.HTML(response.text)

In [40]:
e.xpath('//*[@id="publish"]/div[2]/table/tbody/tr[4]/td/div[2]/strong')

[]

In [83]:
trade = e.xpath('//*[@id="sjl"]')

In [41]:
# response.text